## TODO
1. Lower case tokens DONE
2. Reevaluate 200 to 600 -- DONE....Testing
3. Normalise both dream and fiction to a constant value. Keep the tokens after groupby HALF DONE ....DONE...Testing
<br/> **Issues**: 
    - Check if stopwords have punctuations coz punctuation feature has zero counts. DONE 
      - Punctuation was being removed in the stop word filter function, fixed so that it is included.
    - Suggestion: Add count filter for both dream and fiction?? DONE
    - Normalise based on averge count(instead of max_val) and then normalise to a constant val and add a min-max bound DONE 
      - Edited rescale and get_data to include new constant parameter which specifies if we are using 'avg' or 'max' for normalization

4. Try other algorithms only if Logistic works well -- DONE 
Need a fix for using Naive Bayes, Random Forest, etc -- Link: https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required
5. Add in POS as a feature -- DONE
<br/>**Issue**:
        -Check if Stopword is removing second and third person pronouns like 'he', 'she', 'they'. Coz there are no third person feature in Pronoun and hardly any second person. -- DONE
        -Should we normalise count values again in the feature function?? Removing it tends to give better accuracy -- NOT REQUIRED
6. Look into dream set overfitting, in particular brackets, parantheses, and certain proper names. DONE
<br/> **Issue**: I still see 'ezra' in the unigram feature 
7. Incorporate validation set DONE
8. Test with Confidence Intervals HALF DONE
    - We should repeat this process on different samples of 100 books DONE
        - Edited get_fictional_data to select random samples from Hathi DONE
        - We should maybe brainstorm how to bootstrap and then conduct a bootstrap CI test? **CAN STILL DO THIS**
    - As well as incorporate fiction pages into the validation set for testing contrast
        - Need a better way to access pages to do so, but also is this necessary? I feel like just using the fictional dreams is fine
9. Merge all features into single model DONE
10. Find out why final model is overfitting, add more fictional dreams to the validation dataset HALF DONE
    - Added more fictional dreams, not sure why model is overfitting.
11. Mix & Match Features & models to check if validation accuracy improves --- **IMPORTANT**
    - SVC is promising, gets to around 70-80%. Only issue is that we need a hold out set for fiction data, as a book having already been seen is probably causing higher results DONE
    - Still could mix and match features
12. Check precision, recall, and F1 as well
13. Figure out latex statistics for report
14. Finish off validation set with Akhilesh's dreams and some more from Hathi, goal ~80 dreams

In [2]:
import pandas as pd
import numpy as np
import pickle5 as pickle
import random
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from scipy import sparse
import operator
from tqdm import tqdm
import glob
from htrc_features import FeatureReader, Volume
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.svm import NuSVR
import nltk
import re
import gensim
from gensim import corpora
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

import random

random.seed(1)

[nltk_data] Downloading package stopwords to /home/abc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abc/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
dream_data = pickle.load(open('dreamsDF.pkl', 'rb'))

In [4]:
#Rename chunk in dream_data as page
dream_data = dream_data.rename(columns={"chunk" : "page"})

In [6]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

In [128]:
jockers_list = read_stopwords("../data/jockers.stopwords")

In [7]:
pnouns_set=set()
with open("pronoun.txt", encoding="utf-8") as file:
    for line in file:
        cols=line.rstrip().split("\t")
        pnouns_set.add(cols[0])

final_stopword_set = set(stopwords.words('english')) - pnouns_set
stop_words = {k:1 for k in final_stopword_set}
# stop_words.update(read_stopwords("../data/jockers.stopwords"))
stop_words["dream"]=1
stop_words["dreams"]=1
stop_words["dreamer"]=1
stop_words['dreamt']=1
stop_words['dreamed']=1
stop_words["awake"]=1
stop_words["awaken"]=1
stop_words['awoke']=1
stop_words['sleep']=1
stop_words['-rrb-']=1
stop_words['-lrb-']=1
stop_words['-lsb-']=1
stop_words['-rsb-']=1
stop_words['nan']=1
stop_words['`']=1
stop_words[']']=1
keys_copy = list(stop_words.keys())
for i in keys_copy:
    key = "'"+i
    stop_words[key]=1
stop_words=list(stop_words.keys())

In [8]:
def stop_filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    #has to contain at least one letter or valid punctuation
    if re.search("[a-zA-Z]", word) is not None or re.search('[.!,"`\|:;?\/\\\-{}\']', word) is not None:
        return True
    
    return False

## Finding the average number of tokens in each page of dream and fiction after removing the stop words

In [7]:
stop_filter("'re", stop_words)

False

In [9]:
dream_data = dream_data.loc[[stop_filter(x, stop_words) for x in dream_data['tokens']]] #Also added to the get_data() function
dream_data[dream_data['tokens']=="'re"]

,page,tokens,counts,pos


In [10]:
dream_data.groupby('page').sum().mean()

counts    121.002657
dtype: float64

In [11]:
paths = glob.glob("HathiTrust/fiction"+"/*bz2", recursive=True)
all_vols = FeatureReader(paths)
total_books=0
means=[]
for vol in tqdm(all_vols.volumes()): 
    if(total_books==1):
        break
    total_books+=1
    df = vol.tokenlist().reset_index()
    df.rename(columns={"token":"tokens", "count" : "counts"}, inplace=True)
    df["tokens"]= df["tokens"].str.lower()
    df = df.loc[[stop_filter(x, stop_words) for x in df['tokens']]]
    means.append(df.groupby('page').sum().mean())

sum(means)/len(means)  

AssertionError: 

### We can see that average token counts per page in dream is ~76 and in fiction is ~157. However, there is a huge variance in their max and min
#### Now we need to implement two steps based on this average count
1. Rescale both dream and fiction to 200
2. Remove all the fiction pages which are less than 50 after rescaling. Should we also remove dream ???

In [12]:
dream_data

,page,tokens,counts,pos
1,0,my,4.0,PRP$
3,0,",",10.0,","
5,0,first,1.0,RB
6,0,thing,1.0,NN
7,0,i,12.0,NN
...,...,...,...,...
2478087,26342,celine,2.0,NN
2478088,26342,dion,1.0,NN
2478089,26342,sprinkle,1.0,NN
2478090,26342,oceanside,1.0,NN


## Get Dream Bank data ## DO NOT RUN

data = pd.read_csv("https://raw.githubusercontent.com/josauder/dreambank_visualized/master/dreams.csv")

data.to_csv("DreamBank.csv")

In [11]:
def rescaleCounts(df, rescale_val, constant):
    if constant == 'max':
        grouped_df = df.groupby('page')

        #Get max token count page in the df......We are rescaling based on max counts in the book. We can also use average and have max and min bounds while filtering out
        max_val = grouped_df.sum().max()['counts']

        df_new = grouped_df.apply(lambda x: x['counts']*rescale_val/max_val).reset_index().drop(columns='level_1')
        
    if constant == 'avg':
        grouped_df = df.groupby('page')

        #Get max token count page in the df......We are rescaling based on max counts in the book. We can also use average and have max and min bounds while filtering out
        avg_val = grouped_df.sum().mean()['counts']

        df_new = grouped_df.apply(lambda x: x['counts']*rescale_val/ avg_val).reset_index().drop(columns='level_1')
        
    df_new['tokens'] = df['tokens']
    df_new['pos'] = df['pos']
    df_new.dropna(inplace=True)
    
    return df_new

In [12]:
def get_data(df, label, constant):
    data=[]
    #Filter out the stop_words
    df = df.loc[[stop_filter(x, stop_words) for x in df['tokens']]]
    #Rescale all the tokens to 200
    df = rescaleCounts(df, 200, 'max')
    df['combined']=df['tokens']+'#_'+df['pos']
    ##Create list of feats and label 
    page_list = df.page.unique()
    for page_val in page_list:
        df_filtered = df[df['page']==page_val]
        
        #Filter out page with less than 50 tokens even after rescaling. Should we add this condition for both fiction and dream????
        if constant == 'max':
            if (label==0) and (sum(df_filtered['counts']) < 50):
                continue
            if (label==1) and (sum(df_filtered['counts']) < 0.55):
                continue
        
        #Estimated upper and lower bounds when using average count to normalize (fiction could use some more investigation)
        if constant == 'avg':
            if (label==0) and (sum(df_filtered['counts']) > 300) and (sum(df_filtered['counts']) < 30):
                continue
            if (label==1) and (sum(df_filtered['counts']) > 300) and (sum(df_filtered['counts']) < 30):
                continue
                
        feats= dict(zip(df_filtered['combined'], df_filtered['counts']))
        data.append((feats,label))
        
    return data

In [13]:
#Get fiction data volume
def get_fiction_data(data, fict_folder, total_books, train_test='train'):
    #Get all the volumes

    paths = glob.glob(fict_folder+"/*bz2", recursive=True)
    if train_test == 'train':
        paths = paths[:400] #leaving hold out for test
    if train_test == 'test':
        paths = paths[:-100] #possible volumes for the validation set
    random.shuffle(paths)
    all_vols = FeatureReader(paths)
    
    books=0
    for vol in tqdm(all_vols.volumes()): 
        if(total_books==books):
            break
        books+=1
        df = vol.tokenlist().reset_index()
        df.rename(columns={"token":"tokens", "count" : "counts"}, inplace=True)
        
        df["tokens"]= df["tokens"].str.lower()

        the_list=get_data(df, 0, 'avg')
        data.extend(the_list)

In [15]:
def get_all_data():
    data=[]
    fict_folder= "/home/abc/Berkeley/CompHum/Project/HathiTrust/fiction"
    get_fiction_data(data, fict_folder, 200) #Get 200 books
    data.extend(get_data(dream_data, 1, 'max'))
    #data.extend(get_data(fiction_data,0))
    random.shuffle(data)

    X=[]
    Y=[]
    for unigram_counts, label in data:
        X.append(unigram_counts)
        Y.append(label)

    return X, Y

In [16]:
X, Y=get_all_data()
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

200it [03:10,  1.05it/s]


In [17]:
print(trainY.count(1), trainY.count(0), devY.count(1), devY.count(0))

19934 43208 5095 10691


In [18]:
def build_features(dataX, feature_functions):
    
    """ This function featurizes the data according to the list of parameter feature_functions """
    
    data=[]
    for tokens in dataX:
        feats={}
        
        for function in feature_functions:
            feats.update(function(tokens))

        data.append(feats)
    return data

In [19]:
def features_to_ids(data, feature_vocab):
    
    """ 
    
    This helper function converts a dictionary of feature names to a sparse representation
 that we can fit in a scikit-learn model.  This is important because almost all feature 
 values will be 0 for most documents (note: why?), and we don't want to save them all in 
 memory.

    """
    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    for idx,doc in enumerate(data):
        for f in doc:
            if f in feature_vocab:
                new_data[idx,feature_vocab[f]]=doc[f]
    return new_data

In [20]:
def create_vocab(data, top_n=None):
    
    """ 
    
    This helper function converts a dictionary of feature names to unique numerical ids. 
    top_n limits the features to only the n most frequent features observed in the training data 
    (in terms of the number of documents that contains it).
    
    """
    
    counts=Counter()
    for doc in data:
        for feat in doc:
            counts[feat]+=1

    feature_vocab={}

    for idx, (k, v) in enumerate(counts.most_common(top_n)):
        feature_vocab[k]=idx
                
    return feature_vocab

In [50]:
def pipeline(trainX, devX, trainY, devY, feature_functions, clf):

    """ This function evaluates a list of feature functions on the training/dev data arguments """
    
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data.
    feature_vocab=create_vocab(trainX_feat, top_n=1000)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    ## TO DO: We should build more models with different ML algorithms
#     clf = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000) #Works#Ridge regression
    #clf = linear_model.Perceptron(tol=1e-3, random_state=0) #Works
    #clf=linear_model.LogisticRegression(C=10,penalty='l1', tol=0.01, solver='saga') #Lasso Regression
    #clf = GaussianNB() #Gives error as it needs dense array. Check link: https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required
    #clf=linear_model.BayesianRidge() #Takes too long to run
    #clf= LinearSVC() #Works#Linear Support Vector Classification. Works for both sparse and dense data
    #clf=SVR() #Takes forever to run
    #clf = MultinomialNB() #Works
    #clf = BernoulliNB() #Works
#     clf = RandomForestClassifier(max_depth=2, random_state=0) #Works
    clf.fit(trainX_ids, trainY)
    print("Accuracy: %.3f" % clf.score(devX_ids, devY))
    print("F1-score:", f1_score(devY, clf.predict(devX_ids)))
    
    return clf, feature_vocab

In [22]:
def print_weights(clf, vocab, n=10, model=None):
    if(model=='RF'):
        weights = clf.feature_importances_
    else:
        weights=clf.coef_[0]
    reverse_vocab=[None]*len(weights)
    for k in vocab:
        reverse_vocab[vocab[k]]=k

    for feature, weight in sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))[:n]:
        print("%.3f\t%s" % (weight, feature))

    print()

    for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
        print("%.3f\t%s" % (weight, feature))

In [23]:
def unigram_feature(unigram_counts):

    """ 
    
    See 2.classification/Classification.ipynb for an example feature function to get started.
    This function should return a dictionary with feature names as keys as feature values as values, e.g.:
    {"unigram_the": 0.04, "unigram_fiction": 0.001, ...}
    
    """
    
    feats={}
    total = 0
    # your code goes here
    
    #unigram_counts has count of each word. We first calculate the total number of words
    for word in unigram_counts:
        total+= unigram_counts[word]
        
    #Now we calculate the frequency
    
    for word in unigram_counts:
        word_token = str(word).split('#_')[0]
        feats["unigram_"+str(word_token)] = unigram_counts[word]/total
           
    
    return feats

In [24]:
def pos_feature(unigram_counts): ####TO BE IMPLEMENTED

    """ 
    
    See 2.classification/Classification.ipynb for an example feature function to get started.
    This function should return a dictionary with feature names as keys as feature values as values, e.g.:
    {"unigram_the": 0.04, "unigram_fiction": 0.001, ...}
    
    """
    
    feats={}
    total = 0
    # your code goes here
    
    #unigram_counts has count of each word. We first calculate the total number of words
    for word in unigram_counts:
        total+= unigram_counts[word]
        
    #Now we calculate the frequency
    
    for word in unigram_counts:
        word_pos = str(word).split('#_')[1]
        if("pos_"+str(word_pos) in feats):
            feats["pos_"+str(word_pos)] += unigram_counts[word]/total
        else:
            feats["pos_"+str(word_pos)] = unigram_counts[word]/total
           
    
    return feats

## LIWC

In [25]:
liwc={}
with open("liwc.dictionary.txt", encoding="utf-8") as file:
    for line in file:
        cols=line.rstrip().split("\t")
        word=cols[0]
        liwcs=cols[1].split(" ")
        liwc[word]=liwcs

In [26]:
def liwc_feature(unigram_counts):

    """ 
    
    See 2.classification/Classification.ipynb for an example feature function to get started.
    This function should return a dictionary with feature names as keys as feature values as values, e.g.:
    {"liwc_percept": 0.04, "liwc_affect": 0.05, ...}
    
    """
                
    feats={}


    # your code goes here
    counts=Counter()
    total=0.
    for word in unigram_counts:
        word_token = str(word).split('#_')[0] 
        if word_token in liwc:
            for l in liwc[word_token]:
                counts[l]+=unigram_counts[word]
                total+=unigram_counts[word]
    
    for l in counts:
        feats["liwc_"+str(l)]=float(counts[l])/total
    
    #print(feats)
    return feats
#     return counts

## Pronouns

In [27]:
pnouns={}
with open("pronoun.txt", encoding="utf-8") as file:
    for line in file:
        cols=line.rstrip().split("\t")
        #print(cols)
        word=cols[0]
        pnoun=cols[1].split(" ")
        pnouns[word]=pnoun

In [28]:
def pronoun_feature(unigram_counts):
    feats={}


    # your code goes here
    counts=Counter()
    total=0.
    for word in unigram_counts:
        word_token = str(word).split('#_')[0].lower()
        if word_token in pnouns:
            for l in pnouns[word_token]:
                counts[l]+=unigram_counts[word]
                total+=unigram_counts[word]
    
    for l in counts:
        feats['pnoun'+str(l)]=float(counts[l])/total
    #print(feats)
    return feats

## Punctuation Feature

In [ ]:
##???? Are losing all the punctuations in the stopword filter?? No feature found here.

In [29]:
def punctuation_feature(tokens):
    puncts=[".","!",",",'"',"`","'" ,"|", ":",";","?","/", "\\","-", "{","}","[","]","(",")", "..." ] 
    feats={}


    # your code goes here
    counts=Counter()
    total=0.
    for word in tokens:
        word_token =str(word).split('#_')[0].lower()
        if word_token in puncts:
            counts[word_token]+=1
            total+=1
    
    for l in counts:
        feats['punct'+str(l)]=float(counts[l])/total

    return feats

## Length of words feature

In [30]:
def word_length_feature(tokens):
    max_len = len("pneumonoultramicroscopicsilicovolcanoconiosis") #Longest known word length
    puncts=[".","!",",",'"',"`","'" ,"|", ":",";","?","/", "\\","-", "{","}","[","]","(",")", "..." ] 
    feats={}
    # your code goes here
    counts=Counter()
    total=0.
    for word in tokens:
        word_token =str(word).split('#_')[0].lower()
        if(word_token not in puncts and len(word_token)<= max_len):
            word_len = len(word_token) 
            counts[word_len]+=1
            total+=1
            #print(word_token, word_len)
    for l in counts:
        feats['word_len'+str(l)]=float(counts[l])/total
    
    return feats

## Training a Model

In [62]:
#Read Data
X, Y=get_all_data()
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

400it [06:29,  1.03it/s]


In [31]:
print(trainY.count(1), trainY.count(0), devY.count(1), devY.count(0))

19934 43208 5095 10691


In [ ]:
#clf = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000) #Works#Ridge regression
#clf = linear_model.Perceptron(tol=1e-3, random_state=0) #Works
#clf=linear_model.LogisticRegression(C=10,penalty='l1', tol=0.01, solver='saga') #Lasso Regression
#clf = GaussianNB() #Gives error as it needs dense array. Check link: https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required
#clf=linear_model.BayesianRidge() #Takes too long to run
#clf= LinearSVC(dual=False, max_iter=10000) #Works#Linear Support Vector Classification. Works for both sparse and dense data
#clf=SVR() #Takes forever to run
#clf = MultinomialNB() #Works
#clf = BernoulliNB() #Works
#clf = RandomForestClassifier(max_depth=2, random_state=0) #Works

In [115]:
#Train the model #We can change parameter n to see more dominant features in the list
#features=[unigram_feature, pos_feature, liwc_feature, word_length_feature, punctuation_feature, pronoun_feature]
features=[unigram_feature, pos_feature, liwc_feature]
clf, vocab=pipeline(trainX, devX, trainY, devY, features, clf= RandomForestClassifier(max_depth=15, random_state=0))

Accuracy: 0.986
F1-score: 0.9780765857936276


In [116]:
print_weights(clf, vocab, n=20, model='RF')

0.000	unigram_gently
0.000	unigram_learned
0.000	unigram_noise
0.000	unigram_crossed
0.000	unigram_bar
0.000	unigram_horse
0.000	unigram_repeated
0.000	unigram_send
0.000	unigram_quickly
0.000	unigram_loud
0.000	unigram_busy
0.000	unigram_paid
0.000	unigram_answered
0.000	unigram_forgotten
0.000	unigram_pleased
0.000	unigram_carry
0.000	unigram_finger
0.000	unigram_presence
0.000	unigram_laid
0.000	unigram_closer

0.103	pos_NNP
0.060	pos_VBP
0.054	pos_NN
0.044	pos_VBD
0.025	liwc_shehe
0.023	unigram_he
0.021	pos_JJ
0.020	liwc_certain
0.018	pos_:
0.016	pos_VB
0.016	liwc_you
0.015	liwc_auxverb
0.014	unigram_you
0.014	unigram_i
0.014	liwc_posemo
0.014	pos_UH
0.013	liwc_negemo
0.013	liwc_body
0.013	unigram_it
0.012	unigram_.


## Incorporating Validation Data

In [33]:
validation_dreams = pd.read_csv('validation_dreams.txt', sep='\t')
validation_dreams

,label,id,author,title,text
0,Dream,0,tbd,tbd,"I was in Italy, and awoke (or seemed to awake)..."
1,Dream,1,The Idiot,"Dostoevsky, Fyodor","I was in some room, not my own. It was a large..."
2,Dream,2,War and Peace,"Tolstoy, Leo",I. A. Bazdei'ew is in my house and I want to e...
3,Dream,3,War and Peace,"Tolstoy, Leo","I saw as if in my house in Moscow, Bazdeiew en..."
4,Dream,4,tbd,tbd,some English people had come into the hotel wh...
5,Dream,5,tbd,tbd,"It was a great Piazza, as I thought ; anchored..."
6,Dream,6,Rose o'the sea,"Evans, Marguerite Florence Jervis Barclay","I was climbing great cliffs, like I used to at..."
7,Dream,7,Nightshade,"Gwynne, Paul","I remember vividly every detail of what I saw,..."
8,Dream,8,Yonder,"Young, E.H.",I was walking on a green path and I met a man ...
9,Dream,9,Blind Eyes,"Peterson, Margaret","Ted was dead, she fancied: his white, set face..."


In [34]:
validation_dreams['split'] = validation_dreams['text'].apply(lambda x : nltk.word_tokenize(x.lower()))

In [35]:
validation_dreams['counts'] =validation_dreams.split.apply(lambda x: Counter(x))

In [36]:
validation_dreams = validation_dreams.loc[:,['split','counts']]

In [37]:
validation_dreamsDF = pd.DataFrame.from_records(validation_dreams.counts.values.tolist()).stack().reset_index().rename(columns={'level_0':'page','level_1':'tokens',0:'counts'})

In [38]:
validation_dreamsDF['pos'] = validation_dreamsDF['tokens'].apply(lambda x : (nltk.pos_tag([x]))[0][1])
validation_dreamsDF

,page,tokens,counts,pos
0,0,i,4.0,NN
1,0,was,1.0,VBD
2,0,in,3.0,IN
3,0,italy,1.0,NN
4,0,",",7.0,","
...,...,...,...,...
6577,59,elephant,1.0,NN
6578,59,three-legged,1.0,JJ
6579,59,tub,1.0,NN
6580,59,only—monsieur,1.0,NN


In [39]:
def get_val_all_data():
    data=[]
    fict_folder= "/home/abc/Berkeley/CompHum/Project/HathiTrust/fiction"
    get_fiction_data(data, fict_folder, 1, 'test') #Get 50 books
    data.extend(get_data(validation_dreamsDF, 1, 'avg'))
    #data.extend(get_data(fiction_data,0))
    random.shuffle(data)

    X=[]
    Y=[]
    for unigram_counts, label in data:
        X.append(unigram_counts)
        Y.append(label)

    return X, Y

In [40]:
#Read Validation Data
X_val, Y_val=get_val_all_data()

1it [00:00,  1.92it/s]


In [41]:
print(Y_val.count(1), Y_val.count(0))

60 116


In [73]:
from sklearn.metrics import f1_score
def predict(trainX, trainY, clf, feature_functions):

    """ This function evaluates a list of feature functions on the training/dev data arguments """
    
    trainX_feat=build_features(trainX, feature_functions)
    # just create vocabulary from features in *training* data.
    feature_vocab=create_vocab(trainX_feat, top_n=1000)
    
    trainX_ids=features_to_ids(trainX_feat, feature_vocab)

    accuracy = clf.score(trainX_ids, trainY)
    print("Accuracy: %.3f" % accuracy)
    y_pred = clf.predict(trainX_ids)
    F1 = f1_score(trainY, y_pred)
    print("F1 score:",f1_score(trainY, y_pred))
    return accuracy, F1

In [120]:
acc_all=[]
f1_all=[]
for i in range(100):
    X_val, Y_val=get_val_all_data()
    features=[unigram_feature, pos_feature, liwc_feature]
    acc, f1 = predict(X_val, Y_val, clf, features)
    acc_all.append(acc)
    f1_all.append(f1)

1it [00:01,  1.40s/it]
0it [00:00, ?it/s]

Accuracy: 0.922
F1 score: 0.6923076923076923


1it [00:01,  1.17s/it]
0it [00:00, ?it/s]

Accuracy: 0.925
F1 score: 0.7407407407407408


1it [00:00,  1.26it/s]
0it [00:00, ?it/s]

...

Accuracy: 0.884
F1 score: 0.6285714285714286


1it [00:01,  1.36s/it]
0it [00:00, ?it/s]

Accuracy: 0.910
F1 score: 0.6262626262626263


1it [00:01,  1.20s/it]


Accuracy: 0.901
F1 score: 0.6542056074766355


In [121]:
print("Average accuracy on validation set")
sum(acc_all)/len(acc_all)

Average accuracy on validation set


0.8662172432364713

In [122]:
print("Average F1 score on validation set")
sum(f1_all)/len(f1_all)

Average F1 score on validation set


0.6010240154212846

In [501]:
from math import sqrt 
from scipy.stats import norm

In [568]:
def binomial_confidence_intervals(predictions, truth, confidence_level=0.95):
    correct=[]
    for pred, gold in zip(predictions, truth):
        correct.append(int(pred==gold))
        
    success_rate=np.mean(correct)

    # two-tailed test
    critical_value=(1-confidence_level)/2
    # ppf finds z such that p(X < z) = critical_value
    z_alpha=-1*norm.ppf(critical_value)
    
    # the standard error is the square root of the variance/sample size
    # the variance for a binomial test is p*(1-p)
    standard_error=sqrt((success_rate*(1-success_rate))/len(correct))

    lower=success_rate-z_alpha*standard_error
    upper=success_rate+z_alpha*standard_error
    print("%.3f, %s%% Confidence interval: [%.3f,%.3f]" % (success_rate, confidence_level*100, lower, upper))

In [569]:
binomial_confidence_intervals(preds, Y_val)

0.813, 95.0% Confidence interval: [0.777,0.850]


In [503]:
random.sample(range(1, 200), 10)

[12, 48, 127, 81, 80, 60, 191, 105, 135, 75]

In [504]:
X, Y=get_all_data()
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

2it [00:02,  1.25s/it]


KeyboardInterrupt: 